In [2]:
# Import libraries
import pandas as pd
import json

In [3]:
# Datasets to import
# Tested: May, 21th
URL_BASE = "http://www.deis.cl/wp-content/uploads"
datasets = [
    {
        "source_link": "/2014/01/Defunciones-y-mortalidad-en-la-ni%C3%B1ez-por-regi%C3%B3n-y-comuna-de-residencia.-Chile-2011.xlsx",
        "year": 2011
    },
    {
        "source_link": "/2015/05/7.-Defunciones-y-mortalidad-en-la-ni%C3%B1ez-por-regi%C3%B3n-y-comuna-de-residencia.-Chile-2012.xlsx",
        "year": 2012
    },
    {
        "source_link": "/2016/08/7.-Defunciones-y-mortalidad-en-la-ni%C3%B1ez-por-regi%C3%B3n-y-comuna-de-residencia.-Chile-2013.xlsx",
        "year": 2013
    },
    {
        "source_link": "/2016/08/7.-Defunciones-y-mortalidad-en-la-ni%C3%B1ez-por-regi%C3%B3n-y-comuna-de-residencia.-Chile-2014.xlsx",
        "year": 2014
    }
]

# Read Comunas
URL = "https://raw.githubusercontent.com/datachile/datachile-etl/master/official_ids/2017_06_27_comunas_datachile_fixed.csv"

In [4]:
def search_comuna_id(name):
    name = name.upper().strip().replace(u"\xa0", u" ")
    if name in comuna_datachile_id:
        return comuna_datachile_id[name]
    else:
        return comuna_thesauro_id[name]
    

def get_values(d):
    return [ value for key, value in d.items() ]

In [5]:
geo_datachile_id = pd.read_csv(URL)

comuna_datachile_id = dict(
    zip([elm.upper() for elm in geo_datachile_id["comuna_name"]], geo_datachile_id["comuna_datachile_id"])
)

age_range = pd.read_csv("https://docs.google.com/spreadsheets/d/" + 
                   "152kELSLYe4oiCiH7-Wmt-FWvSYavT0LUQt1C_qXnXVM" +
                   "/export?gid=0&format=csv"
                  )

age_range.to_csv("data/age_range_id.csv", index=False)

formatted_df = pd.melt(age_range, id_vars=["id"], value_vars=["es"])
q = formatted_df.to_dict()

age_range_ids = dict(zip(get_values(q["value"]), get_values(q["id"])))

thesauro_comunas = pd.read_csv("https://docs.google.com/spreadsheets/d/" + 
                   "1ccZd25Q5BfeL2w-KCbe1k7ZJIuskJa0gwoTZsillBGk" +
                   "/export?gid=0&format=csv"
                  )

comuna_thesauro_id = dict(zip([elm.upper() for elm in thesauro_comunas["nombre"]], thesauro_comunas["id"]))

In [12]:
data = pd.read_excel(URL_BASE + datasets[0]["source_link"])

# Read title/subtitle of Excel
title = [ str(item).strip() for item in data.iloc[2] ]
subtitle = [ str(item).strip() for item in data.iloc[3] ]

In [23]:
uuu = [(1), (2), (3), (4), (5)]
print(uuu[2:])

[3, 4, 5]


In [33]:
tidy = []

for dataset in datasets:
    data = pd.read_excel(URL_BASE + dataset["source_link"])
    
    # Read title/subtitle of Excel
    title = [ str(item).strip() for item in data.iloc[2] ]
    subtitle = [ str(item).strip() for item in data.iloc[3] ]
    measures = [ str(item).strip() for item in data.iloc[4] ]
    
    output = []
    for i in range(5, len(data) - 2):
        frame = data.iloc[i]
        #tuples = list(zip(title, subtitle, measures, frame))
        
        geo = { "comuna": str(frame[1]).strip() } if str(frame[0]).strip() == "nan" else { "country": str(frame[0]).strip() }  if str(frame[0]).strip() == "Total País" else { "region": str(frame[0]).strip() } 
        #tuples = list(filter(lambda x: x[0] != "nan", tuples))
        
        age_range = ""
        sex = ""
        measure = ""
        
        tuples = []
        for tup in list(zip(title, subtitle, measures, frame))[2:]:
            if tup[0] != "nan":
                age_range = tup[0]
            if tup[1] != "nan":
                sex = tup[1]
            if tup[2] != "nan":
                measure = tup[2]
            tuples.append((age_range, sex, measure, tup[3]))
        
        age_range = ""
        
        query = []
        for key, tup in enumerate(tuples):
            if tup[2] != "Tasa*":
                age_range = tup[0]
                count = tup[3]

            if key % 2 != 0:
                item = {
                    "age_range": age_range,
                    "count": count,
                    "sex": tup[1],
                    "rate": tup[3]
                }
                query.append(item)
        
        tuples = {**geo, "children": query}
        output.append(tuples)
        
    # Process data
    country_data = {}
    region_data = {}
    for key, item in enumerate(output):
        if "country" in item:
            country_data = { d["age_range"]: d["rate"] for d in item["children"] }

        if "region" in item:
            region_data = { d["age_range"]: d["rate"] for d in item["children"] }

        if "comuna" in item:
            for d in item["children"]:
                out = {
                    "comuna_id": search_comuna_id(item["comuna"]),
                    "year": dataset["year"],
                    #"age_range": d["age_range"],
                    "age_range_id": age_range_ids[d["age_range"]],
                    "sex": d["sex"],
                    "rate_country": country_data[d["age_range"]],
                    "rate_region": region_data[d["age_range"]],
                    "rate_comuna": 0 if d["rate"] == "-" else d["rate"],
                    "count": 0 if d["count"] == "-" else d["count"]
                }
                tidy.append(out)

        

{'age_range': '1 a 4 años', 'count': 151, 'sex': 'Hombres', 'rate': 0.29693254911677314}
{'age_range': '1 a 4 años', 'count': 129, 'sex': 'Mujeres', 'rate': 0.26330077112427386}
{'age_range': '5 a 9 años', 'count': 89, 'sex': 'Hombres', 'rate': 0.14101668274890633}
{'age_range': '5 a 9 años', 'count': 70, 'sex': 'Mujeres', 'rate': 0.11504680761544125}
{'age_range': '1 a 4 años', 'count': 3, 'sex': 'Hombres', 'rate': 0.5488474204171241}
{'age_range': '1 a 4 años', 'count': 1, 'sex': 'Mujeres', 'rate': 0.18814675446848542}
{'age_range': '5 a 9 años', 'count': 2, 'sex': 'Hombres', 'rate': 0.28914269191846176}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 3, 'sex': 'Hombres', 'rate': 0.5576208178438662}
{'age_range': '1 a 4 años', 'count': 1, 'sex': 'Mujeres', 'rate': 0.19116803670426305}
{'age_range': '5 a 9 años', 'count': 2, 'sex': 'Hombres', 'rate': 0.29364263691087944}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres',

{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 2, 'sex': 'Hombres', 'rate': 1.5936254980079683}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 1, 'sex': 'Hombres', 'rate': 0.4110152075626798}
{'age_range': '1 a 4 años', 'count': 1, 'sex': 'Mujeres', 'rate': 0.4308487720809996}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 1, 'sex': 'Mujeres', 'rate': 0.3465003465003465}
{'age_range': '1 a 4 años', 'count'

{'age_range': '1 a 4 años', 'count': 296, 'sex': 'Ambos sexos', 'rate': 29.57911714329683}
{'age_range': '1 a 4 años', 'count': 162, 'sex': 'Hombres', 'rate': 31.784337384855352}
{'age_range': '1 a 4 años', 'count': 134, 'sex': 'Mujeres', 'rate': 27.290075169901083}
{'age_range': '5 a 9 años', 'count': 177, 'sex': 'Ambos sexos', 'rate': 14.255087133213385}
{'age_range': '5 a 9 años', 'count': 100, 'sex': 'Hombres', 'rate': 15.81755400508376}
{'age_range': '5 a 9 años', 'count': 77, 'sex': 'Mujeres', 'rate': 12.634280248025687}
{'age_range': '1 a 4 años', 'count': 8, 'sex': 'Ambos sexos', 'rate': 75.52157084867366}
{'age_range': '1 a 4 años', 'count': 4, 'sex': 'Hombres', 'rate': 74.50176941702365}
{'age_range': '1 a 4 años', 'count': 4, 'sex': 'Mujeres', 'rate': 76.5696784073507}
{'age_range': '5 a 9 años', 'count': 1, 'sex': 'Ambos sexos', 'rate': 7.423353871279043}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 1, 'sex': 'Muj

{'age_range': '1 a 4 años', 'count': 2, 'sex': 'Ambos sexos', 'rate': 45.248868778280546}
{'age_range': '1 a 4 años', 'count': 1, 'sex': 'Hombres', 'rate': 44.62293618920125}
{'age_range': '1 a 4 años', 'count': 1, 'sex': 'Mujeres', 'rate': 45.892611289582376}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 8, 'sex': 'Ambos sexos', 'rate': 14.928436806060946}
{'age_range': '1 a 4 años', 'count': 2, 'sex': 'Hombres', 'rate': 7.340796476417691}
{'age_range': '1 a 4 años', 'count': 6, 'sex': 'Mujeres', 'rate': 22.775584573337383}
{'age_range': '5 a 9 años', 'count': 4, 'sex': 'Ambos sexos', 'rate': 6.056293245718957}
{'age_range': '5 a 9 años', 'count': 3, 'sex': 'Hombres', 'rate': 8.957897879964168}
{'age_range': '5 a 9 años', 'count': 1, 'sex': 'Mujeres', 'rate': 3.071536075191203}
{'age_ra

{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 2, 'sex': 'Ambos sexos', 'rate': 111.23470522803115}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 2, 'sex': 'Mujeres', 'rate': 227.7904328018223}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Ho

...

{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 1, 'sex': 'Mujeres', 'rate': 24.691358024691358}
{'age_range': '1 a 4 años', 'count': 1, 'sex': 'Ambos sexos', 'rate': 3.487966515521451}
{'age_range': '1 a 4 años', 'count': 1, 'sex': 'Hombres', 'rate': 7.178750897343862}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0

{'age_range': '1 a 4 años', 'count': 283, 'sex': 'Ambos sexos', 'rate': 2.853918692562365}
{'age_range': '1 a 4 años', 'count': 160, 'sex': 'Hombres', 'rate': 3.1663355207434556}
{'age_range': '1 a 4 años', 'count': 123, 'sex': 'Mujeres', 'rate': 2.52928729619188}
{'age_range': '5 a 9 años', 'count': 166, 'sex': 'Ambos sexos', 'rate': 1.3812430521810328}
{'age_range': '5 a 9 años', 'count': 96, 'sex': 'Hombres', 'rate': 1.5650932045609425}
{'age_range': '5 a 9 años', 'count': 70, 'sex': 'Mujeres', 'rate': 1.1895981537436653}
{'age_range': '1 a 4 años', 'count': 12, 'sex': 'Ambos sexos', 'rate': 8.145533532446375}
{'age_range': '1 a 4 años', 'count': 7, 'sex': 'Hombres', 'rate': 9.303561935140882}
{'age_range': '1 a 4 años', 'count': 5, 'sex': 'Mujeres', 'rate': 6.936736958934517}
{'age_range': '5 a 9 años', 'count': 1, 'sex': 'Ambos sexos', 'rate': 0.5835667600373483}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 1, 'sex': 'Mu

{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 2, 'sex': 'Ambos sexos', 'rate': 11.46788990825688}
{'age_range': '1 a 4 años', 'count': 2, 'sex': 'Hombres', 'rate': 22.471910112359552}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ho

{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 1, 'sex': 'Ambos sexos', 'rate': 8.045052292839904}
{'age_range': '5 a 9 años', 'count': 1, 'sex': 'Hombres', 'rate': 15.08295625942685}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 7, 'sex': 'Ambos sexos', 'rate': 7.835236176404746}
{'age_range': '1 a 4 años', 'count': 2, 'sex': 'Hombres', 'rate': 4.388852315119596}
{'age_range': '1 a 4 añ

{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': '-'}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Hombres', 'rate': '-'}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': '-'}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': '-'}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': '-'}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': '-'}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '1 a 4 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Ambos sexos', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Hombres', 'rate': 0}
{'age_range': '5 a 9 años', 'count': 0, 'sex': 'Mujeres', 'rate': 0}
{'age_

In [29]:
print(tidy)


[{'comuna_id': 12, 'year': 2011, 'age_range_id': 5, 'sex': 'Hombres', 'rate_country': 0.26330077112427386, 'rate_region': 0.18814675446848542, 'rate_comuna': 0.5576208178438662, 'count': 0.5576208178438662}, {'comuna_id': 12, 'year': 2011, 'age_range_id': 5, 'sex': 'Mujeres', 'rate_country': 0.26330077112427386, 'rate_region': 0.18814675446848542, 'rate_comuna': 0.19116803670426305, 'count': 0.19116803670426305}, {'comuna_id': 12, 'year': 2011, 'age_range_id': 6, 'sex': 'Hombres', 'rate_country': 0.11504680761544125, 'rate_region': 0, 'rate_comuna': 0.29364263691087944, 'count': 0.29364263691087944}, {'comuna_id': 12, 'year': 2011, 'age_range_id': 6, 'sex': 'Mujeres', 'rate_country': 0.11504680761544125, 'rate_region': 0, 'rate_comuna': 0, 'count': 0}, {'comuna_id': 25, 'year': 2011, 'age_range_id': 5, 'sex': 'Hombres', 'rate_country': 0.26330077112427386, 'rate_region': 0.18814675446848542, 'rate_comuna': 0, 'count': 0}, {'comuna_id': 25, 'year': 2011, 'age_range_id': 5, 'sex': 'Mujer

In [13]:
# In 2011
print(title)
print(subtitle)

['Región y comuna', 'nan', '1 a 4 años', 'nan', 'nan', 'nan', '5 a 9 años', 'nan', 'nan', 'nan']
['nan', 'nan', 'Hombres', 'nan', 'Mujeres', 'nan', 'Hombres', 'nan', 'Mujeres', 'nan']
